In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import seaborn as sns
 
from sklearn.model_selection import train_test_split, ShuffleSplit, learning_curve, GridSearchCV, KFold, StratifiedKFold
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.metrics import roc_curve, accuracy_score, confusion_matrix, classification_report, roc_auc_score, make_scorer, precision_recall_curve, average_precision_score 
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import metrics

In [ ]:
def N_unique_values(df):
    return np.array([len(set([i for i in x[~pd.isnull(x)]])) for x in df.values])

In [ ]:
!pip install catboost

In [ ]:
!pip install shap

In [ ]:
from sklearn.ensemble import RandomForestClassifier, IsolationForest, VotingClassifier
from sklearn.neural_network import MLPClassifier

%matplotlib inline
plt.style.use('ggplot')

from catboost import CatBoostClassifier # Or CatBoostRegressor
from sklearn.model_selection import KFold
from itertools import product,chain

import shap
import catboost
from catboost import *
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import recall_score, accuracy_score, confusion_matrix, classification_report
import random

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


np.random.seed(0)
random.seed(0)

In [ ]:
from sortedcontainers import SortedList
import copy
import collections
import numpy as np
from itertools import product,chain
import pandas
from sklearn.model_selection import KFold
import catboost as cb

In [ ]:
class paramsearch:
    def __init__(self,pdict):    
        self.pdict = {}
        # if something is not passed in as a sequence, make it a sequence with 1 element
        #   don't treat strings as sequences
        for a,b in pdict.items():
            if isinstance(b, collections.Sequence) and not isinstance(b, str): self.pdict[a] = b
            else: self.pdict[a] = [b]
        # our results are a sorted list, so the best score is always the final element
        self.results = SortedList()       
                    
    def grid_search(self,keys=None):
        # do grid search on only the keys listed. If none provided, do all
        if keys==None: keylist = self.pdict.keys()
        else: keylist = keys
        listoflists = [] # this will be list of lists of key,value pairs
        for key in keylist: listoflists.append([(key,i) for i in self.pdict[key]])
        for p in product(*listoflists):
            # do any changes to the current best parameter set
            if len(self.results)>0: template = self.results[-1][1]
            else: template = {a:b[0] for a,b in self.pdict.items()}
            # if our updates are the same as current best, don't bother
            if self.equaldict(dict(p),template): continue
            # take the current best and update just the ones to change
            yield self.overwritedict(dict(p),template)
                              
    def equaldict(self,a,b):
        for key in a.keys(): 
            if a[key] != b[key]: return False
        return True            

    def overwritedict(self,new,old):
        old = copy.deepcopy(old)
        for key in new.keys(): old[key] = new[key]
        return old            
    
    # save a (score,params) pair to results. Since 'results' is a sorted list,
    #   the best score is always the final element. A small amount of noise is added
    #   because sorted lists don't like it when two scores are exactly the same    
    def register_result(self,result,params):
        self.results.add((result+np.random.randn()*1e-10,params))    
        
    def bestscore(self):
        return self.results[-1][0]
        
    def bestparam(self):
        return self.results[-1][1]


#Read data

In [ ]:
# Function to load the inputs dataset
def read_data(tp = "Train", N = 1542865627584):
    target = pd.read_csv("/content/drive/MyDrive/Data/Kaggledata/train/Train-1542865627584.csv".format(tp.title(), N))
    pt = pd.read_csv("/content/drive/MyDrive/Data/Kaggledata/train/Train_Beneficiarydata-1542865627584.csv".format(tp.title(), N))
    in_pt = pd.read_csv("/content/drive/MyDrive/Data/Kaggledata/train/Train_Inpatientdata-1542865627584.csv".format(tp.title(), N))
    out_pt = pd.read_csv("/content/drive/MyDrive/Data/Kaggledata/train/Train_Outpatientdata-1542865627584.csv".format(tp.title(), N))
    return (in_pt, out_pt, pt, target)

In [ ]:
in_pt, out_pt, ben, target = read_data()

In [ ]:
ben

#Data Cleaning

In [ ]:
# Replace values with a binary annotation
ben = ben.replace({'ChronicCond_Alzheimer': 2, 'ChronicCond_Heartfailure': 2, 'ChronicCond_KidneyDisease': 2,
                   'ChronicCond_Cancer': 2, 'ChronicCond_ObstrPulmonary': 2, 'ChronicCond_Depression': 2,
                   'ChronicCond_Diabetes': 2, 'ChronicCond_IschemicHeart': 2, 'ChronicCond_Osteoporasis': 2,
                   'ChronicCond_rheumatoidarthritis': 2, 'ChronicCond_stroke': 2, 'Gender': 2 }, 
                  0)
ben = ben.replace({'RenalDiseaseIndicator': 'Y'}, 1).astype({'RenalDiseaseIndicator': 'int64'})

# Change target variable to binary
target["target"] = np.where(target.PotentialFraud == "Yes", 1, 0) 
target.drop('PotentialFraud', axis=1, inplace=True)

In [ ]:
# Merge in_pt, out_pt and ben df into a single patient dataset
data = pd.merge(in_pt, out_pt,
                    left_on = [ idx for idx in out_pt.columns if idx in in_pt.columns],
                    right_on = [ idx for idx in out_pt.columns if idx in in_pt.columns],
                    how = 'outer').\
          merge(ben,left_on='BeneID',right_on='BeneID',how='inner')

In [ ]:
data

In [ ]:
patient_merge_id = [i for i in out_pt.columns if i in in_pt.columns]

# Merge in_pt, out_pt and ben df into a single patient dataset
data = pd.merge(in_pt, out_pt,
                    left_on = patient_merge_id,
                    right_on = patient_merge_id,
                    how = 'outer').\
          merge(ben,left_on='BeneID',right_on='BeneID',how='inner')

In [ ]:
data

#Feature Engineering

In [ ]:
# We find the number of unique physicians 
data['N_unique_Physicians'] = N_unique_values(data[['AttendingPhysician', 'OperatingPhysician', 'OtherPhysician']]) 

# We separate the types of physicians into numeric values
data[['AttendingPhysician', 'OperatingPhysician', 'OtherPhysician']] = np.where(data[['AttendingPhysician','OperatingPhysician',
                                                                                      'OtherPhysician']].isnull(), 0, 1)

# We count the number of types of physicians that attend the patient
data['N_Types_Physicians'] = data['AttendingPhysician'] +  data['OperatingPhysician'] + data['OtherPhysician']

# Now we create a variable to check if there is a single doctor on a patient that was attended by more than 1 type of doctor
# This helps us finds those cases that are only looked at by 1 physicians
data['Same_Physician'] = data.apply(lambda x: 1 if (x['N_unique_Physicians'] == 1 and x['N_Types_Physicians'] > 1) else 0,axis=1)

# Similar to Same_Physician, we create a variable to see if 1 physicians has had multiple roles, but has not been alone reviewing the case
data['Same_Physician2'] = data.apply(lambda x: 1 if (x['N_unique_Physicians'] == 2 and x['N_Types_Physicians'] > 2) else 0,axis=1)

# We check our new variables
data[['N_unique_Physicians','N_Types_Physicians','Same_Physician','Same_Physician2']].head()

In [ ]:
# We count the number of procedures for each claim, we drop the initial variables
ClmProcedure_vars = ['ClmProcedureCode_{}'.format(x) for x in range(1,7)]
data['N_Procedure'] = N_unique_values(data[ClmProcedure_vars])
data = data.drop(ClmProcedure_vars, axis = 1)

# We count the number of claims, we also separate this by unique claims and extra claims, we drop the initial variables
ClmDiagnosisCode_vars =['ClmAdmitDiagnosisCode'] + ['ClmDiagnosisCode_{}'.format(x) for x in range(1, 11)]

data['N_Unique_Claims'] = N_unique_values(data[ClmDiagnosisCode_vars])
data['N_Total_Claims'] = data[ClmDiagnosisCode_vars].notnull().to_numpy().sum(axis = 1)
data['N_Extra_Claims'] = data['N_Total_Claims'] - data['N_Unique_Claims']

ClmDiagnosisCode_vars.append('N_Total_Claims')
data = data.drop(ClmDiagnosisCode_vars, axis = 1)

In [ ]:
#  Transform string columns of date into type date
data['AdmissionDt'] = pd.to_datetime(data['AdmissionDt'] , format = '%Y-%m-%d')
data['DischargeDt'] = pd.to_datetime(data['DischargeDt'],format = '%Y-%m-%d')

data['ClaimStartDt'] = pd.to_datetime(data['ClaimStartDt'] , format = '%Y-%m-%d')
data['ClaimEndDt'] = pd.to_datetime(data['ClaimEndDt'],format = '%Y-%m-%d')

data['DOB'] = pd.to_datetime(data['DOB'] , format = '%Y-%m-%d')
data['DOD'] = pd.to_datetime(data['DOD'],format = '%Y-%m-%d')

# Number of days
data['Admission_Days'] = ((data['DischargeDt'] - data['AdmissionDt']).dt.days) + 1

# Number of claim days 
data['Claim_Days'] = ((data['ClaimEndDt'] - data['ClaimStartDt']).dt.days) + 1

# Age at the time of claim
data['Age'] = round(((data['ClaimStartDt'] - data['DOB']).dt.days + 1)/365.25)

In [ ]:
# We create a Hospitalization flag 
data['Hospt'] = np.where(data.DiagnosisGroupCode.notnull(), 1, 0)
data = data.drop(['DiagnosisGroupCode'], axis = 1)

# Variable if patient is dead
data['Dead']= 0
data.loc[data.DOD.notna(),'Dead'] = 1

#Handling Miss Data

In [ ]:
# We find which variables hold missing data
na = data.isnull().sum()
na[na != 0]

In [ ]:
## We know that missing admission days come from missing admission and discharge date, and those cases come from the out patients data set


# We also see that there are some cases of missing deductible amount paid, so we also want to keep an eye on that
data['Missing_Deductible_Amount_Paid'] = 0
data.loc[data['DeductibleAmtPaid'].isnull(), 'Missing_Deductible_Amount_Paid'] = 1 

# After identifying the missing values, we fill the missing values with 0
data = data.fillna(0).copy()

In [ ]:
data

#Create Final Dataset

In [ ]:
### Sum all numeric variables
_sum = data.groupby(['Provider'], as_index = False)[['InscClaimAmtReimbursed', 'DeductibleAmtPaid', 'RenalDiseaseIndicator', 
                                                    'ChronicCond_Alzheimer', 'AttendingPhysician', 'OperatingPhysician', 
                                                    'OtherPhysician', 'N_unique_Physicians', 'ChronicCond_Heartfailure', 
                                                    'N_Types_Physicians', 'Same_Physician',
                                                    'ChronicCond_KidneyDisease', 'ChronicCond_Cancer', 
                                                    'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression', 
                                                    'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart', 
                                                    'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis',
                                                    'ChronicCond_stroke', 'Dead', 
                                                    'N_Procedure','N_Unique_Claims', 'N_Extra_Claims', 'Admission_Days',
                                                    'Claim_Days', 'Hospt', 'Missing_Deductible_Amount_Paid']].sum()

# To separate our variables, we shall add '_sum' at the end of their names
_sum = _sum.add_suffix('_sum')


### Count number of records
_count = data[['BeneID', 'ClaimID']].groupby(data['Provider']).nunique().reset_index()
_count.rename(columns={'BeneID':'BeneID_count','ClaimID':'ClaimID_count'},inplace=True)

### Calculate mean for all numeric variables
_mean = data.groupby(['Provider'], as_index = False)[['NoOfMonths_PartACov', 'NoOfMonths_PartBCov',
                                                      'IPAnnualReimbursementAmt', 'IPAnnualDeductibleAmt',
                                                      'OPAnnualReimbursementAmt', 'OPAnnualDeductibleAmt', 'Age',
                                                      'AttendingPhysician', 'OperatingPhysician','OtherPhysician',
                                                      'N_unique_Physicians', 'ChronicCond_Heartfailure', 
                                                      'N_Types_Physicians', 'Same_Physician',
                                                      'ChronicCond_KidneyDisease', 'ChronicCond_Cancer', 
                                                      'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression', 
                                                      'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart', 
                                                      'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis',
                                                      'ChronicCond_stroke', 'Dead', 'N_Procedure','N_Unique_Claims', 
                                                      'N_Extra_Claims', 'Admission_Days','Claim_Days', 'Hospt',
                                                      'Missing_Deductible_Amount_Paid']].mean()

# To separate our variables, we shall add '_mean' at the end of their names
_mean = _mean.add_suffix('_mean')

# We create a dataset that holds all the variables
_total = _count.merge(_sum, how='left',left_on='Provider',right_on='Provider_sum').\
                merge(_mean, how='left',left_on='Provider',right_on='Provider_mean').\
                drop(['Provider_sum','Provider_mean'], axis=1).\
                merge(target, on='Provider', how='left')


In [ ]:
_total.groupby( ["target"] ).target.count().plot(kind = "bar", figsize = (10,6));

#Feature Selection - Correlation

In [ ]:
def plot_corr(df_corr):
    corrMatrix = df_corr.corr()
    plt.subplots(figsize=(20,15))
    sns.heatmap(corrMatrix, annot=False)
    plt.show()

In [ ]:
# We plot the correlations in the _sum dataset
plot_corr(_sum)

In [ ]:
# We drop correlated values 
sum_corr = _sum.drop(['Hospt_sum','AttendingPhysician_sum','OperatingPhysician_sum','Admission_Days_sum',
                      'OtherPhysician_sum','ChronicCond_ObstrPulmonary_sum', 'ChronicCond_Depression_sum',
                      'ChronicCond_Diabetes_sum','ChronicCond_IschemicHeart_sum','ChronicCond_KidneyDisease_sum', 
                      'ChronicCond_Cancer_sum','ChronicCond_Osteoporasis_sum','RenalDiseaseIndicator_sum',
                      'ChronicCond_rheumatoidarthritis_sum','ChronicCond_Heartfailure_sum','N_Unique_Claims_sum',
                      'ChronicCond_Alzheimer_sum','ChronicCond_stroke_sum','N_Procedure_sum','N_unique_Physicians_sum',
                      'N_Types_Physicians_sum','DeductibleAmtPaid_sum'],axis=1) 

# And plot again to check
plot_corr(sum_corr)

In [ ]:
# We plot the correlations in the _mean dataset
plot_corr(_mean)

In [ ]:
# We drop correlated values 
mean_corr = _mean.drop(['Hospt_mean','Admission_Days_mean','N_unique_Physicians_mean','NoOfMonths_PartBCov_mean',
                        'NoOfMonths_PartACov_mean','IPAnnualReimbursementAmt_mean','N_Procedure_mean',
                        'OPAnnualDeductibleAmt_mean', 'N_Unique_Claims_mean'], axis=1) 

# And plot again to check
plot_corr(mean_corr)

#Unsupervised Model 

In [ ]:
!pip install pyod

In [ ]:
from pyod.models.pca import PCA

In [ ]:
clf_mean_PCA = PCA()
clf_mean_PCA
#_mean, _sum
clf_mean_PCA.fit(_mean.iloc[:,1:31])
label_mean_PCA = clf_mean_PCA.predict(_mean.iloc[:,1:31], return_confidence=False)
label_mean_PCA #541

In [ ]:
from pyod.models.iforest import 

In [ ]:
clf_mean_IForest = IForest()
clf_mean_IForest
#_mean, _sum
clf_mean_IForest.fit(_mean.iloc[:,1:31])
label_mean_IForest = clf_mean_IForest.predict(_mean.iloc[:,1:31], return_confidence=False)
label_mean_IForest #541

In [ ]:
from pyod.models.auto_encoder import AutoEncoder
clf_mean_AE = AutoEncoder(epochs=30, hidden_neurons =[2, 1, 1, 2])
clf_mean_AE
#_mean, _sum
clf_mean_AE.fit(_mean.iloc[:,1:31])
label_mean_AE = clf_mean_AE.predict(_mean.iloc[:,1:31], return_confidence=False)
label_mean_AE #541

#Combine label with data

In [ ]:
mean_data_PCA = pd.concat([_mean, pd.DataFrame(label_mean_PCA)],axis=1)

In [ ]:
mean_data_IForest = pd.concat([_mean, pd.DataFrame(label_mean_IForest)],axis=1)

In [ ]:
mean_data_AE = pd.concat([_mean, pd.DataFrame(label_mean_AE)],axis=1)

In [ ]:
combined = pd.concat([_mean, pd.DataFrame(label_mean_PCA), pd.DataFrame(label_mean_IForest), pd.DataFrame(label_mean_AE)],axis=1)

In [ ]:
label_combined = combined.iloc[:,32:35].sum(axis=1)

In [ ]:
label_combined

In [ ]:
from collections import Counter

In [ ]:
Counter(label_combined)

#和真实label比较

In [ ]:
#mean_data_PCA，mean_data_IForest，mean_data_AE

In [ ]:
label_comp = pd.concat([_total['target'],pd.DataFrame(label_mean_PCA), pd.DataFrame(label_mean_IForest), pd.DataFrame(label_mean_AE), label_combined],axis =1)

In [ ]:
label_comp

In [ ]:
#PCA:
print(sum(label_comp.iloc[:,0] == label_comp.iloc[:,1]) / len(label_comp.iloc[:,0]))
print(sum(label_comp.iloc[:,0] + label_comp.iloc[:,1] == 2) / sum(label_comp.iloc[:,0]==1))
print(sum(label_comp.iloc[:,0]==1))
print(sum(label_comp.iloc[:,1] == 1))

In [ ]:
#IF
print(sum(label_comp.iloc[:,0] == label_comp.iloc[:,2]) / len(label_comp.iloc[:,0]))
print(sum(label_comp.iloc[:,0] + label_comp.iloc[:,2] == 2) / sum(label_comp.iloc[:,0]==1))
print(sum(label_comp.iloc[:,0]==1))
print(sum(label_comp.iloc[:,2] == 1))

In [ ]:
#IF
print(sum(label_comp.iloc[:,0] == label_comp.iloc[:,3]) / len(label_comp.iloc[:,0]))
print(sum(label_comp.iloc[:,0] + label_comp.iloc[:,3] == 2) / sum(label_comp.iloc[:,0]==1))
print(sum(label_comp.iloc[:,0]==1))
print(sum(label_comp.iloc[:,3] == 1))


In [ ]:
#data.apply(lambda x: 1 if (x['N_unique_Physicians'] == 1 and x['N_Types_Physicians'] > 1) else 0,axis=1)
array = []
for i in range(len(label_comp.iloc[:,0])):
  if label_comp.iloc[i,4] >= 1:
    array.append(1)
  else:
    array.append(0)
label_comp1 = pd.concat([label_comp, pd.DataFrame(array)],axis = 1)

In [ ]:
#combined
print(sum(label_comp1.iloc[:,5] == label_comp1.iloc[:,0]) / len(label_comp1.iloc[:,0]))
print(sum(label_comp1.iloc[:,5] + label_comp1.iloc[:,0] == 2) / len(label_comp1.iloc[:,0]))
print(sum(label_comp1.iloc[:,5] + label_comp1.iloc[:,0] == 2) / sum(label_comp1.iloc[:,0]==1))

#XGBoost

## Data set

In [ ]:
PCA_prob = clf_mean_PCA.predict_proba(_mean.iloc[:,1:31], method='linear', return_confidence=True)[0][:,0]
IF_prob = clf_mean_IForest.predict_proba(_mean.iloc[:,1:31], method='linear', return_confidence=True)[0][:,0]
AE_prob = clf_mean_AE.predict_proba(_mean.iloc[:,1:31], method='linear', return_confidence=True)[0][:,0]

In [ ]:
_total_ = _count.merge(_mean, how='left',left_on='Provider',right_on='Provider_mean').\
                drop(['Provider_mean'], axis=1).\
                merge(target, on='Provider', how='left')

In [ ]:
total_su = pd.concat([_total_.iloc[:,1:34],pd.DataFrame(PCA_prob),pd.DataFrame(IF_prob),pd.DataFrame(AE_prob),_total_.iloc[:,-1]],axis=1)

In [ ]:
#total_su

In [ ]:
plot_corr(total_su)

In [ ]:
total_su

In [ ]:
#total_su = pd.concat([total_su.iloc[:,0:35],total_su.iloc[:,36]],axis=1)

In [ ]:
plot_corr(total_su)

##Shap Value

In [ ]:
#total_su
df = _total[['Provider','InscClaimAmtReimbursed_sum','N_Extra_Claims_sum','Claim_Days_sum',
             'AttendingPhysician_mean','Missing_Deductible_Amount_Paid_mean','Dead_mean','Claim_Days_mean','N_Extra_Claims_mean',
             'BeneID_count','ClaimID_count',
             'target']]


In [ ]:
df

In [ ]:
df_su = pd.concat([df.iloc[:,0:11],pd.DataFrame(PCA_prob),pd.DataFrame(IF_prob),df.iloc[:,11]], axis = 1)

## Model

### Carboost

In [ ]:
def cm_Score(model, df):
    recall = np.mean(cross_val_score(model, df.drop(['Provider','target'], axis = 1), df.target, cv=3, scoring='recall'))
    accuracy = np.mean(cross_val_score(model, df.drop(['Provider','target'], axis = 1), df.target, cv=3, scoring='accuracy'))
    
    print('Accuracy Score: {}'.format(accuracy))
    print('Recall Score: {}'.format(recall))

In [ ]:
#model, X_train, X_test, y_train, y_test = fcatboost(df_su.drop(['Provider','target'], axis = 1), df_su.target)

###Logistic regression

In [ ]:
def flogistic(df, penalty):
    X_train, X_test, y_train, y_test = train_test_split(df_su.drop(['Provider','target'], axis = 1),df.target, 
                                                  test_size=0.30, random_state=1)

    X_train = pd.DataFrame(StandardScaler().fit_transform(X_train))
    X_test = pd.DataFrame(StandardScaler().fit_transform(X_test))
    
    #Liblinear for small datasets of binary classes
    logreg = LogisticRegression(penalty= penalty,solver= 'liblinear',class_weight='balanced', random_state = 5 , C = 0.001)
    
    cm_Score(logreg, df_su)
    

In [ ]:
flogistic(df_su, 'l1')

In [ ]:
flogistic(df_su, 'l2')

##Naive Bayes

In [ ]:
def Gaus_bayes(df):
    X_train, X_test, y_train, y_test = train_test_split(df.drop(['Provider','target'], axis = 1),df.target, 
                                                  test_size=0.30, random_state=1)


    X_train = pd.DataFrame(StandardScaler().fit_transform(X_train))
    X_test = pd.DataFrame(StandardScaler().fit_transform(X_test))

    gnb = GaussianNB()
    
    cm_Score(gnb, df)

In [ ]:
Gaus_bayes(df_su)

##XGBoost

In [2]:
!pip install pyod

     |████████████████████████████████| 113 kB 30.4 MB/s 
  Created wheel for pyod: filename=pyod-0.9.5-py3-none-any.whl size=132699 sha256=971196f93257036ec77ebc868292fe1d8859efa28b5f2fef3ba0899652ad2429
  Stored in directory: /root/.cache/pip/wheels/3d/bb/b7/62b60fb451b33b0df1ab8006697fba7a6a49709a629055cf77
Successfully built pyod


In [3]:
from pyod.models.xgbod import XGBOD
from pyod.models.iforest import IForest
from pyod.models.pca import PCA

In [4]:
estimators = [IForest(),PCA()]
clf_mean_XGBOD = XGBOD(estimator_list=estimators, standardization_flag_list=None, max_depth=3, learning_rate=0.1, 
                       n_estimators=100, silent=True, objective='binary:logistic', booster='gbtree', n_jobs=1, nthread=None, 
                       gamma=0, min_child_weight=1, max_delta_step=0, subsample=1, colsample_bytree=1, colsample_bylevel=1, 
                       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, base_score=0.5, random_state=0,)

In [5]:
clf_mean_XGBOD

XGBOD(base_score=0.5, booster='gbtree', colsample_bylevel=1,
   colsample_bytree=1,
   estimator_list=[IForest(behaviour='old', bootstrap=False, contamination=0.1, max_features=1.0,
    max_samples='auto', n_estimators=100, n_jobs=1, random_state=None,
    verbose=0), PCA(contamination=0.1, copy=True, iterated_power='auto', n_components=None,
  n_selected_components=None, random_state=None, standardization=True,
  svd_solver='auto', tol=0.0, weighted=True, whiten=False)],
   gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
   min_child_weight=1, n_estimators=100, n_jobs=1, nthread=None,
   objective='binary:logistic', random_state=0, reg_alpha=0, reg_lambda=1,
   scale_pos_weight=1, silent=True, standardization_flag_list=None,
   subsample=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_su.drop(['Provider','target'], axis = 1),df.target, 
                                                  test_size=0.30, random_state=1)

X_train = pd.DataFrame(StandardScaler().fit_transform(X_train))
X_test = pd.DataFrame(StandardScaler().fit_transform(X_test))
#y_train = pd.DataFrame(StandardScaler().fit_transform(y_train))
#y_test = pd.DataFrame(StandardScaler().fit_transform(y_test))  

#Liblinear for small datasets of binary classes
clf_mean_xgbod = clf_mean_XGBOD.fit(X_train,y_train)
predict_xgbod = clf_mean_xgbod.predict(X_test)
    
#cm_Score(clf_mean_XGBOD, df_su)

In [ ]:
len(y_test),len(predict_xgbod)

In [ ]:
sum(y_test == predict_xgbod)/1623